In [1]:
# Import libraries 
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
# For connect to google sheet
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
# Configure the connection 
scope = ['https://spreadsheets.google.com/feeds']
# Give the path to the Service Account Credential json file 
credentials = ServiceAccountCredentials.from_json_keyfile_name('eltos-delight-f8d3c6a963ea.json',scope)
# Authorise your Notebook
gc = gspread.authorize(credentials)
# The sprad sheet ID, which can be taken from the link to the sheet
spreadsheet_key = '1QqCQk3hIHiJ_v_3ITtKBtJidcPEaHDCflXvMgxosVbA'
book = gc.open_by_key(spreadsheet_key) 
worksheet = book.worksheet("tio") 
table = worksheet.get_all_values()

In [2]:
##Convert table data into a dataframe 
df = pd.DataFrame(table[1:], columns=table[0])
##Convert number strings to floats and ints 
df = df.apply(pd.to_numeric, errors='ignore')

In [3]:
df['Full Name'] = df['First Name'] + ' '+ df['Last Name'] 
mid = df['Full Name']
df.drop(labels=['Full Name'], axis=1, inplace = True)
df.insert(1, 'Full Name', mid)
df['Full Name']= df['Full Name'].str.title()

In [4]:
#lets all column name start with capital letter
for col in df:
    col.title()

In [5]:
df['State of Residence'] = df['State of Residence'].str.title()
df['State of Residence'] = df[['State of Residence']].applymap(lambda x: x.split()[0]+ ' State')

In [6]:
df = df.rename(columns= {'Timestamp' : 'Date'})
df = df.rename(columns= {'What flavor' : 'Flavours'})
import datetime
import calendar

In [7]:
df['Date']= pd.to_datetime(df['Date'])
df.columns

Index(['Date', 'Full Name', 'First Name', 'Last Name', 'Phone number', 'Email',
       'Address', 'State of Residence', 'Size', 'Flavours',
       'Social Media Handle(s)'],
      dtype='object')

In [8]:
def category(ss):
    if ss['Size'] == '900g':
        return 2
    elif ss['Size'] == '430g':
        return 1
    else:
        return 3
    

In [9]:
df['Unit'] = df.apply(category, axis=1)

In [10]:
df['Amount'] = df['Unit'] * 750
df.columns

Index(['Date', 'Full Name', 'First Name', 'Last Name', 'Phone number', 'Email',
       'Address', 'State of Residence', 'Size', 'Flavours',
       'Social Media Handle(s)', 'Unit', 'Amount'],
      dtype='object')

In [11]:
for_analysis = df[['Date','Full Name','State of Residence','Size', 'Flavours','Unit','Amount']]

In [12]:
from df2gspread import df2gspread as d2g 
wks_name = 'For_Analysis' 
d2g.upload(for_analysis, spreadsheet_key, wks_name, credentials=credentials, row_names=True)



<Worksheet 'For_Analysis' id:506361202>